In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ReadParquetExample") \
    .getOrCreate()


In [4]:
df = spark.read.parquet("parquet/20250323/part-00000-70719350-f19b-4f49-9c97-36dcad7803bd-c000.snappy.parquet")

In [6]:
df.printSchema()   # shows column names and types
# Show 5 rows, do not truncate any column, and print each row vertically
df.show(5, truncate=False, vertical=True)


root
 |-- SOURCEURL: string (nullable = true)
 |-- first(SQLDATE): string (nullable = true)
 |-- EventCount: long (nullable = true)
 |-- GLOBALEVENTIDs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- GoldsteinMean: double (nullable = true)
 |-- GoldsteinMedian: double (nullable = true)
 |-- MedianPGoldstein: double (nullable = true)
 |-- ToneMean: double (nullable = true)
 |-- MedianAvgTone: double (nullable = true)
 |-- EventCodes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Cameos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- All_descriptions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ActorPairs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Actor1Name: string (nullable = true)
 |    |    |-- Actor2Name: string (nullable = true)
 |-- GeoPairs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 

In [8]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum, when

# Build a combined predicate: any column IS NULL
any_null = None
for c in df.columns:
    is_null_c = col(c).isNull()
    any_null = is_null_c if any_null is None else any_null | is_null_c

# Count rows matching it
rows_with_nulls = df.filter(any_null).count()
print(f"Rows with at least one null: {rows_with_nulls}")



Rows with at least one null: 0


In [10]:
df.filter(any_null).limit(10).show(truncate=False, vertical=True)


(0 rows)



In [11]:
# 1) Number of rows
row_count = df.count()
print(f"Total rows: {row_count}")

# 2) Number of columns
num_columns = len(df.columns)
print(f"Total columns: {num_columns}")


Total rows: 14168
Total columns: 14


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct, desc

cols = df.columns

summary_rows = []
for c in cols:
    non_null = df.filter(col(c).isNotNull()).count()
    distinct = df.select(countDistinct(c)).first()[0]
    top = (
        df.groupBy(c)
          .count()
          .orderBy(desc("count"))
          .limit(1)
          .collect()
    )
    if top:
        top_val, top_freq = top[0][c], top[0]["count"]
        # **coerce to string** so every top_val is a Python str
        top_val = str(top_val)
    else:
        top_val, top_freq = None, 0

    summary_rows.append((c, non_null, distinct, top_val, top_freq))

# Now Spark can infer a consistent schema: all top_val fields are strings
summary_df = spark.createDataFrame(
    summary_rows,
    schema=["column", "non_null", "distinct", "top_value", "top_freq"]
)
summary_df.show(truncate=False)


+----------------+--------+--------+----------------------------------------------------------------------------+--------+
|column          |non_null|distinct|top_value                                                                   |top_freq|
+----------------+--------+--------+----------------------------------------------------------------------------+--------+
|SOURCEURL       |14168   |14168   |http://www.darientimes.com/opinion/article/tong-ui-risk-utility-20236533.php|1       |
|first(SQLDATE)  |14168   |6       |20250323                                                                    |13536   |
|EventCount      |14168   |69      |1                                                                           |3650    |
|GLOBALEVENTIDs  |14168   |14168   |['1233752629', '1233752640', '1233752641']                                  |1       |
|GoldsteinMean   |14168   |3508    |0.0                                                                         |747     |
|GoldsteinMedian

In [19]:
from pyspark.sql.functions import col

# Rename the column so it’s easier to refer to:
unique_dates_df = df.select(col("first(SQLDATE)").alias("SQLDATE")).distinct().orderBy("SQLDATE")       # optional: sort them chronologically

unique_dates_df.show(truncate=False)


+--------+
|SQLDATE |
+--------+
|20150326|
|20240323|
|20250221|
|20250316|
|20250322|
|20250323|
+--------+



In [20]:
from pyspark.sql.functions import regexp_extract, col

# This regex grabs the part between “://” (optional) and the next “/” (or end-of-string).
df2 = df.withColumn(
    "domain",
    regexp_extract(
        col("SOURCEURL"),
        r'^(?:https?://)?(?:www\.)?([^/]+)',
        1
    )
)

df2.select("SOURCEURL", "domain").show(truncate=False)


+----------------------------------------------------------------------------------------------------------------+-----------------------+
|SOURCEURL                                                                                                       |domain                 |
+----------------------------------------------------------------------------------------------------------------+-----------------------+
|http://baystreet.ca/articles/economiccommentary/109783/Canadian-PM-Mark-Carney-Expected-To-Call-Snap-Election   |baystreet.ca           |
|http://businessghana.com/site/news/general/324856/WaterAid-demands-action-to-tackle-crisis                      |businessghana.com      |
|http://businessghana.com/site/news/general/324857/Inculcate-Japanese-edu-style-in-schools                       |businessghana.com      |
|http://businessghana.com/site/news/general/324858/ECOWAS-Court-Conducts-Preparatory-Meeting                     |businessghana.com      |
|http://colombogazette.com/

In [21]:
df2.printSchema()   # shows column names and types
# Show 5 rows, do not truncate any column, and print each row vertically
df2.show(5, truncate=False, vertical=True)

root
 |-- SOURCEURL: string (nullable = true)
 |-- first(SQLDATE): string (nullable = true)
 |-- EventCount: long (nullable = true)
 |-- GLOBALEVENTIDs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- GoldsteinMean: double (nullable = true)
 |-- GoldsteinMedian: double (nullable = true)
 |-- MedianPGoldstein: double (nullable = true)
 |-- ToneMean: double (nullable = true)
 |-- MedianAvgTone: double (nullable = true)
 |-- EventCodes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Cameos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- All_descriptions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ActorPairs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- Actor1Name: string (nullable = true)
 |    |    |-- Actor2Name: string (nullable = true)
 |-- GeoPairs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 

In [28]:
from pyspark.sql.functions import count

# 1) Cache and trigger an action
df2.cache()
df2.count()               # materializes the entire DF in cache

# 2) Now overwrite the same location
df2.write.mode("overwrite") \
   .parquet("parquet/20250323")


Py4JJavaError: An error occurred while calling o851.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 368.0 failed 1 times, most recent failure: Lost task 0.0 in stage 368.0 (TID 233) (np-light-indigo-populus-db8d798cf-sbsv5 executor driver): org.apache.spark.SparkFileNotFoundException: File file:/home/jovyan/shared/Taher/parquet/20250323/part-00000-70719350-f19b-4f49-9c97-36dcad7803bd-c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources